


---


#Universidad Politécnica Salesiana
##Alumno: Rayner Palta
###Materia: Simulación

---


##Al modelo de simulación de la venta de boletos de un teatro/cine agregar la simulación grafica 2D/3D utilizando cualquier libreria grafica, por ejemplo pygame/tkinter, etc. generar un informe en formato PDF que me permita evidenciar la simulación y generar una grafica/reporte de los resultados.




In [2]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.


In [40]:
import collections
import random
import simpy


##--------------------------------------------------------------------------------------------------------------------------------------------------------


def ventaBoletos(env, num_boletos, pelicula, teatro):
  with teatro.contador.request() as turno:
    resultado = yield turno | teatro.sold_out[pelicula]
    if turno not in resultado:
      teatro.num_renegados[pelicula] += 1
      return
    if teatro.num_boletos[pelicula] < num_boletos:
      yield env.timeout(0.5) 
      return
    teatro.num_boletos[pelicula] -= num_boletos
    if teatro.num_boletos[pelicula] < 2:
      teatro.sold_out[pelicula].succeed()
      teatro.tiempo_agotado[pelicula] = env.now
      teatro.num_boletos[pelicula] = 0
    yield env.timeout(1)


def llegadaClientes(env, teatro):
  while True:
    yield env.timeout(random.expovariate(1/0.5))
    pelicula = random.choices(teatro.peliculas, teatro.probabilidad, k=1)
    num_boletos = random.randint(1, 10)
    if teatro.num_boletos[pelicula[0]]:
      env.process(ventaBoletos(env, num_boletos, pelicula[0], teatro))


def process_run(velocidas,cantidadBoleto,tmSimulacion):
    
    VELOCIDAD_LLEGADA = velocidas
    NUM_BOLETO = cantidadBoleto
    TIEMPO_SIMULACION = tmSimulacion
    
    resultado = ""
    
    Teatro = collections.namedtuple('Teatro', 'contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados')

    print('Teatro Carlos Crespi - UPS')
    
    resultado = "Teatro Carlos Crespi - UPS"
    env = simpy.Environment()

    contador = simpy.Resource(env,capacity=1)
    peliculas = ['Conjuro 3', 'Rapidos y Furiosos 10', 'Pulp Fictions', 'La liga de la Justicia', 'Godzilla vs Kong']
    
    resultado = resultado +"\n"
    
    probabilidad=[0.1, 0.3, 0.5, 0.7, 0.9]
    num_boletos = {pelicula: NUM_BOLETO for pelicula in peliculas}
    sold_out = {pelicula: env.event() for pelicula in peliculas}
    tiempo_agotado = {pelicula: None for pelicula in peliculas}
    num_renegados = {pelicula: 0 for pelicula in peliculas}

    teatro = Teatro(contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados)
    env.process(llegadaClientes(env, teatro))
    env.run(until=TIEMPO_SIMULACION)

    # Analisis y resultados
    for pelicula in peliculas:
        if teatro.sold_out[pelicula]:
            print('Pelicula: %s se agoto en el tiempo %.1f despues de salir a la venta' %(pelicula, teatro.tiempo_agotado[pelicula]))
            sms1 = 'Pelicula: %s se agoto en el tiempo %.1f despues de salir a la venta' %(pelicula, teatro.tiempo_agotado[pelicula])
            
            print('Numero de personas que salieron de la fila/renegados %s' %teatro.num_renegados[pelicula])
            sms2 = 'Numero de personas que salieron de la fila/renegados %s' %teatro.num_renegados[pelicula]
            
            resultado = resultado + "\n" + sms1 +"\n"+ sms2
    return resultado

In [4]:
resp = process_run(vel=52,nboleto=18,tsimulacion=290)
print("Resultado=> ",resp)

Teatro Carlos Crespi - UPS
Pelicula: Conjuro 3 se agoto en el tiempo 76.1 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 20.9 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Pulp Fictions se agoto en el tiempo 13.8 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 2
Pelicula: La liga de la Justicia se agoto en el tiempo 15.8 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Godzilla vs Kong se agoto en el tiempo 17.3 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Resultado=>  Teatro Carlos Crespi - UPS

Pelicula: Conjuro 3 se agoto en el tiempo 76.1 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 20.9 despues de salir a la venta
Numero de personas que salier

In [5]:
from tkinter import *
from tkinter import messagebox
from tkinter import ttk
import tkinter as tk



In [41]:
grafica = tk.Tk()
grafica.geometry("500x500")
grafica.title("Simulación de boleteria peliculas")
grafica.config(bg = 'powder blue')
frame =Frame (bg = 'black')
grafica.resizable(0,0)

txtVelocidad=StringVar()
velocidad = Label(grafica,text="Velocidad", bg='gray', fg='black', width=10).place(x=15, y=10)
lblVelocidad = ttk.Entry(grafica, textvariable=txtVelocidad).place(x=150,y=10)

txtBoleto=StringVar()
cantidadBoleto = Label( grafica,text="Cantida boletos",  bg='gray', fg='black',width=20).place(x=5, y=40)
lblBoleto = ttk.Entry(grafica, textvariable=txtBoleto).place(x=150,y=40)

txtSimulacion=StringVar()
simulacion = Label(grafica,text="Tiempo de simulacion",  bg='gray', fg='black',width=20).place(x=10, y=70)
lblSimulacion = ttk.Entry(grafica,textvariable=txtSimulacion).place(x=150,y=70)


def guardar():
    velocida = txtVelocidad.get()
    boletos=txtBoleto.get()
    simulacions=txtSimulacion.get()
    
    resultado = process_run(velocidas=int(velocida), cantidadBoleto =int(boletos),tmSimulacion=int(simulacions))
    print("resp ",resultado)
    messagebox.showinfo(message=resultado,title="Datos")
    

Button(grafica, text="Ejecutar",  bg='gray', fg='black', command=guardar).place(x=100, y=150)

grafica.mainloop()

Teatro Carlos Crespi - UPS
Pelicula: Conjuro 3 se agoto en el tiempo 32.5 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/rayner/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-41-4c1a87a93967>", line 26, in guardar
    resultado = process_run(velocidas=int(velocida), cantidadBoleto =int(boletos),tmSimulacion=int(simulacions))
  File "<ipython-input-40-c2b4e474e637>", line 68, in process_run
    print('Pelicula: %s se agoto en el tiempo %.1f despues de salir a la venta' %(pelicula, teatro.tiempo_agotado[pelicula]))
TypeError: must be real number, not NoneType


Teatro Carlos Crespi - UPS
Pelicula: Conjuro 3 se agoto en el tiempo 50.7 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 51.7 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Pulp Fictions se agoto en el tiempo 20.4 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: La liga de la Justicia se agoto en el tiempo 16.4 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 8
Pelicula: Godzilla vs Kong se agoto en el tiempo 12.4 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 2
resp  Teatro Carlos Crespi - UPS

Pelicula: Conjuro 3 se agoto en el tiempo 50.7 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 51.7 despues de salir a la venta
Numero de personas que salieron de l